In [42]:
import os
import asyncio
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [26]:
load_dotenv()
model = ChatOpenAI(
    model="gpt-5-nano",  # or "gpt-4o" for better quality
    temperature=0,
    max_tokens=256,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [43]:
system_prompt = """
You are an email summarizer. Write ONE concise paragraph including these fields exactly once:
Email_id, Thread_id, From, To, CC, Source, Date, Labels, Subject.

Rules:
- Keep all details EXACTLY as given (names, emails, dates, labels, subject).
- Use full lists for To and CC, no 'etc.' or omissions.
- If a field is empty or None, write "None".
- Use proper grammar, single paragraph only.

Example:
Email abc123, part of thread xyz456, was sent by John Doe (john@example.com). The To recipients are jane@example.com and mark@example.com. The CC's are boss@example.com and hr@example.com. It originated from Gmail on 2024-08-20. The email carries the labels Important and Work. The subject of the email is 'Project Update'.
"""

In [44]:
async def get_summaries(metadata_list: list[str]) -> list[str]:
    # Prepare messages
    requests = [
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Email metadata:\n{m}"}
        ]
        for m in metadata_list
    ]

    # Use async batch
    responses = await model.abatch(requests)

    summaries = []
    for res in responses:
        summaries.append(res.content.strip())

    return summaries

In [48]:
metadata_samples = [
        """Email_id: eml987654
Thread_id: thr123456
From: Alice Johnson (alice.johnson@example.com)
To: bob.smith@example.com, carol.white@example.com, dave.miller@example.com
CC: erin.green@example.com, frank.brown@example.com, george.king@example.com
Source: Microsoft Outlook
Date: 2024-09-15
Labels: High Priority, Client Communication, Follow-Up
Subject: Quarterly Financial Review and Next Steps""",

        """Email_id: eml543210
Thread_id: thr678910
From: Michael Roberts (michael.roberts@example.com)
To: natalie.lee@example.com, oliver.james@example.com, sophia.clark@example.com, thomas.walker@example.com
CC: victoria.hall@example.com, william.young@example.com, xavier.turner@example.com
Source: Gmail
Date: 2024-08-25
Labels: Confidential, Internal Review, Strategy Planning
Subject: Upcoming Product Launch Strategy and Marketing Alignment"""
]

In [49]:
results = await get_summaries(metadata_samples)

In [50]:
print(results)

['', '']
